In [1]:
print(1)
import os
from datetime import datetime
from collections import defaultdict 
import numpy as np

{'review_id': 'lWC-xP3rd6obsecCYsGZRg', 'user_id': 'ak0TdVmGKo4pwqdJSTLwWw', 'business_id': 'buF9druCkbuXLX526sGELQ', 'stars': 4.0, 'useful': 3, 'funny': 1, 'cool': 1, 'text': "Apparently Prides Osteria had a rough summer as evidenced by the almost empty dining room at 6:30 on a Friday night. However new blood in the kitchen seems to have revitalized the food from other customers recent visits. Waitstaff was warm but unobtrusive. By 8 pm or so when we left the bar was full and the dining room was much more lively than it had been. Perhaps Beverly residents prefer a later seating. \n\nAfter reading the mixed reviews of late I was a little tentative over our choice but luckily there was nothing to worry about in the food department. We started with the fried dough, burrata and prosciutto which were all lovely. Then although they don't offer half portions of pasta we each ordered the entree size and split them. We chose the tagliatelle bolognese and a four cheese filled pasta in a creamy 

AttributeError: module 'datetime' has no attribute 'fromisoformat'

In [11]:
data = defaultdict(list)
i_stat = defaultdict(int)

with open('./yelp/yelp_academic_dataset_review.json') as f:
    for line in f:
        r = eval(line)
        u = r['user_id']
        i = r['business_id']
        t = int(datetime.fromisoformat(r['date']).timestamp())
        data[u].append((i, int(t)))
        i_stat[i] += 1


# Dataset stat

In [12]:
print('# user', len(data))
print('# item', len(i_stat))

behavior_len = [len(v) for k, v in data.items()]
total_num = sum(behavior_len)
print('# records', total_num)

max_u_len = np.max(behavior_len)
min_u_len = np.min(behavior_len)
avg_u_len = np.mean(behavior_len)
med_u_len = np.median(behavior_len)

print('max user behavior seq len: ', max_u_len)
print('min user behavior seq len: ', min_u_len)
print('avg user behavior seq len: ', avg_u_len)
print('median user behavior seq len: ', med_u_len)


i_u_len = [v for k, v in i_stat.items()]
max_i_len = np.max(i_u_len)
min_i_len = np.min(i_u_len)
avg_i_len = np.mean(i_u_len)
med_i_len = np.median(i_u_len)

print('max item behavior seq len: ', max_i_len)
print('min item behavior seq len: ', min_i_len)
print('avg item behavior seq len: ', avg_i_len)
print('median item behavior seq len: ', med_i_len)


max_len_uid = 0
for k, v in data.items():
    if len(v) >= max_u_len:
        max_len_uid = k
        break
        
# id2u = {v:k for k, v in u_vocab.items()}
# id2i = {v:k for k, v in i_vocab.items()}
print('the max seq len user: ', max_len_uid)

valid_user_20 = {k:v for k, v in data.items() if len(v) > 20}
print('# users with len > 20: ', len(valid_user_20))

valid_user_40 = {k:v for k, v in data.items() if len(v) > 40}
print('# users with len > 40: ', len(valid_user_40))

valid_user_20_2k = {k:v for k, v in data.items() if len(v) > 20 and len(v) < 2000}
print('# users with 2000 > len > 20: ', len(valid_user_20_2k))

valid_user_40_2k = {k:v for k, v in data.items() if len(v) > 40 and len(v) < 2000}
print('# users with 2000 > len > 40: ', len(valid_user_40_2k))

valid_user_20_1k = {k:v for k, v in data.items() if len(v) > 20 and len(v) < 1000}
print('# users with 1000 > len > 20: ', len(valid_user_20_1k))

valid_user_40_1k = {k:v for k, v in data.items() if len(v) > 40 and len(v) < 1000}
print('# users with 1000 > len > 40: ', len(valid_user_40_1k))

valid_item_5 = {k:v for k, v in i_stat.items() if v > 5}
print('# item with freq > 5: ', len(valid_item_5))

print('# records with 1000 > seq len > 40', sum([len(v) for k, v in valid_user_40_1k.items()]))

print('# records with 2000 > seq len > 40', sum([len(v) for k, v in valid_user_40_2k.items()]))


# user 2189457
# item 160585
# records 8635403
max user behavior seq len:  6073
min user behavior seq len:  1
avg user behavior seq len:  3.944084309488608
median user behavior seq len:  1.0
max item behavior seq len:  9295
min item behavior seq len:  5
avg item behavior seq len:  53.77465516704549
median item behavior seq len:  17.0
the max seq len user:  RtGqdDBvvBCjcu5dUqwfzA
# users with len > 20:  56580
# users with len > 40:  22272
# users with 2000 > len > 20:  56577
# users with 2000 > len > 40:  22269
# users with 1000 > len > 20:  56557
# users with 1000 > len > 40:  22249
# item with freq > 5:  148425
# records with 1000 > seq len > 40 2079235
# records with 2000 > seq len > 40 2105083


# Clean data

In [13]:
# valid_data = copy.deepcopy(data)
data = {k:v for k, v in data.items() if len(v) > 40 and len(v) < 2000}

## item > 5,  2000 > # user > 40
while True:
    i_stat = defaultdict(int)
    for u, hist in data.items():
        for i, t in hist:
            i_stat[i] += 1

    i_stat_new = {i:l for i, l in i_stat.items() if l > 5}
    
    for u, hist in data.items():
        data[u] = [(i, t) for i, t in hist if i in i_stat_new]
    data_new = {u:hist for u, hist in data.items() if len(hist) > 40}
    
    if len(data_new) == len(data) and len(i_stat) == len(i_stat_new):
        break
    else:
        data = data_new


# data stat

print('# user', len(data))
print('# item', len(i_stat))

behavior_len = [len(v) for k, v in data.items()]
total_num = sum(behavior_len)
print('# records', total_num)

max_u_len = np.max(behavior_len)
min_u_len = np.min(behavior_len)
avg_u_len = np.mean(behavior_len)
med_u_len = np.median(behavior_len)

print('max user behavior seq len: ', max_u_len)
print('min user behavior seq len: ', min_u_len)
print('avg user behavior seq len: ', avg_u_len)
print('median user behavior seq len: ', med_u_len)


i_u_len = [v for k, v in i_stat.items()]
max_i_len = np.max(i_u_len)
min_i_len = np.min(i_u_len)
avg_i_len = np.mean(i_u_len)
med_i_len = np.median(i_u_len)

print('max item behavior seq len: ', max_i_len)
print('min item behavior seq len: ', min_i_len)
print('avg item behavior seq len: ', avg_i_len)
print('median item behavior seq len: ', med_i_len)


valid_user_20 = {k:v for k, v in data.items() if len(v) > 20}
print('# users with len > 20: ', len(valid_user_20))

valid_user_40 = {k:v for k, v in data.items() if len(v) > 40}
print('# users with len > 40: ', len(valid_user_40))

valid_user_20_2k = {k:v for k, v in data.items() if len(v) > 20 and len(v) < 2000}
print('# users with 2000 > len > 20: ', len(valid_user_20_2k))

valid_user_40_2k = {k:v for k, v in data.items() if len(v) > 40 and len(v) < 2000}
print('# users with 2000 > len > 40: ', len(valid_user_40_2k))

valid_user_20_1k = {k:v for k, v in data.items() if len(v) > 20 and len(v) < 1000}
print('# users with 1000 > len > 20: ', len(valid_user_20_1k))

valid_user_40_1k = {k:v for k, v in data.items() if len(v) > 40 and len(v) < 1000}
print('# users with 1000 > len > 40: ', len(valid_user_40_1k))

valid_item_5 = {k:v for k, v in i_stat.items() if v > 5}
print('# item with freq > 5: ', len(valid_item_5))



# user 19150
# item 58962
# records 1791721
max user behavior seq len:  1786
min user behavior seq len:  41
avg user behavior seq len:  93.56245430809399
median user behavior seq len:  66.0
max item behavior seq len:  1158
min item behavior seq len:  6
avg item behavior seq len:  30.38772429700485
median item behavior seq len:  15.0
# users with len > 20:  19150
# users with len > 40:  19150
# users with 2000 > len > 20:  19150
# users with 2000 > len > 40:  19150
# users with 1000 > len > 20:  19134
# users with 1000 > len > 40:  19134
# item with freq > 5:  58962


# save data

In [14]:
u_vocab = {}
i_vocab = {}
for u, hist in data.items():
    if u not in u_vocab:
        u_vocab[u] = len(u_vocab) + 1
    for i, t in hist:
        if i not in i_vocab:
            i_vocab[i] = len(i_vocab) + 1

import json
output = {
        'user_vocab': u_vocab,
        'item_vocab': i_vocab
        }
    
with open('./yelp/yelp_vocab.json', 'w') as f:
    json.dump(output, f, indent=4)

max([v for k, v in u_vocab.items()]), max([v for k, v in i_vocab.items()])

data = {u_vocab[u]: [(i_vocab[i], t) for i, t in seq] for u, seq in data.items()}

# sorted by timestamp
data = {k:sorted(v, key=lambda x: x[1]) for k, v in data.items()}

# write file
with open(os.path.join('./yelp/', 'yelp.csv'), 'w') as f:
    f.write('user_id\titem_id\ttime\n')
    for u, seq in data.items():
        for i, t in seq:
             f.write('\t'.join(map(str, [u, i, t]))+'\n')

# Analysis data percentage, timestamp, and test item converageå

In [3]:
def build_init_data_by_ts(train, ratio):
    full_seq = []
    for u, seq in train.items():
        full_seq += seq

    full_seq = sorted(full_seq, key=lambda x: x[1])
    idx = int(len(full_seq) * ratio + 0.5)
    timestamp = full_seq[idx - 1][1]

    init_train = {}
    for u, seq in train.items():
        init_train[u] = [x for x in seq if x[1] < timestamp]
    
    return init_train, timestamp


def build_init_data_by_ts_and_k(train, ratio, init_k):
    total_num = sum([len(seq) for u, seq in train.items()])

    init_data = {u: seq[: init_k] for u, seq in train.items()}
    remain_data = {u: seq[init_k:] for u, seq in train.items()}
    
    idx = int(total_num * ratio + 0.5) - init_k * len(train)
    
    remain_seq = []
    for u, seq in remain_data.items():
        remain_seq += seq

    remain_seq = sorted(remain_seq, key=lambda x: x[1])
    timestamp = remain_seq[idx - 1][1]
    
    init_train = {}
    for u, seq in remain_data.items():
        init_train[u] = init_data[u] + [x for x in seq if x[1] < timestamp]

    return init_train, timestamp


def init_data_stat(init_train, train):
    init_item_set = set()
    init_empty_user = []
    for u, seq in init_train.items():
        init_item_set = init_item_set.union(set([x[0] for x in seq]))
        if len(seq) == 0:
            init_empty_user.append(u)

    behavior_len = [len(v) for k, v in init_train.items()]

    print('# init train :', sum(behavior_len))
    print('# data ratio :', 1.0 * sum(behavior_len) / sum([len(seq) for u, seq in train.items()]))
    print('# empty user in train number:', len(init_empty_user))

    max_u_len = np.max(behavior_len)
    min_u_len = np.min(behavior_len)
    avg_u_len = np.mean(behavior_len)
    med_u_len = np.median(behavior_len)
    print('max user behavior seq len: ', max_u_len)
    print('min user behavior seq len: ', min_u_len)
    print('avg user behavior seq len: ', avg_u_len)
    print('median user behavior seq len: ', med_u_len)     

    print('====================== test item coverage =====================')
    print('# items in init training data: ', len(init_item_set))  
    coverage = 0.
    lost_num = 0

    for u, x in test.items():
        if x[0] in init_item_set:
            coverage += 1
        else:
            lost_num += 1
    print('test coverage in train: ', coverage, coverage / (coverage + lost_num))
    print('# test items not found in training data: ', lost_num)


In [4]:
data = defaultdict(list)
with open(os.path.join('./yelp/', 'yelp.csv')) as f:
    next(f)
    for line in f:
        u, i, t = line.strip().split('\t')
        data[int(u)].append((int(i), int(t)))

user_clicked_set = {}
full_traim_num = 0
train, test = {}, {}
for u, seq in data.items():
    seq = sorted(seq, key=lambda x: x[1])
    train[u], test[u] = seq[:-1], seq[-1]
    user_clicked_set[u] = set([x[0] for x in train[u]])
    full_traim_num += len(train[u])

train_item_set = set()
for u, seq in train.items():
    train_item_set = train_item_set.union(set([x[0] for x in seq]))

coverage = 0.
lost_num = 0

for u, x in test.items():
    if x[0] in train_item_set:
        coverage += 1
    else:
        lost_num += 1
        
coverage /= len(test)
print('test coverage in train: ', coverage)
print('# test items not found in training data: ', lost_num)

test coverage in train:  0.9993211488250653
# test items not found in training data:  13


In [5]:
print('# full train :', sum([len(seq) for u, seq in train.items()]))

# 30% init data
ratio = 0.3
print('================ build data only using timestamp ================')
init_train, timestamp = build_init_data_by_ts(train, ratio)
print('timestamp {} for {} training data: '.format(timestamp, ratio))
init_data_stat(init_train, train)

print('\nbuild data using timestamp and make sure each user has at least 5 behaviors')
init_train, timestamp = build_init_data_by_ts_and_k(train, ratio, 5)
print('timestamp {} for {} and init 5 training data: '.format(timestamp, ratio))
init_data_stat(init_train, train)

# full train : 1772571
================ build data only using timestamp ================
timestamp 1340034361 for 0.3 training data: 
# init train : 531770
# data ratio : 0.2999992666020148
# empty user in train number: 9164
max user behavior seq len:  940
min user behavior seq len:  0
avg user behavior seq len:  27.768668407310706
median user behavior seq len:  1.0
====================== test item coverage =====================
# items in init training data:  37565
test coverage in train:  8941.0 0.4668929503916449
# test items not found in training data:  10209

build data using timestamp and make sure each user has at least 5 behaviors
timestamp 1328108739 for 0.3 and init 5 training data: 
# init train : 531770
# data ratio : 0.2999992666020148
# empty user in train number: 0
max user behavior seq len:  866
min user behavior seq len:  5
avg user behavior seq len:  27.768668407310706
median user behavior seq len:  5.0
====================== test item coverage =====================
#

In [6]:
print('# full train :', sum([len(seq) for u, seq in train.items()]))

# 30% init data
ratio = 0.5
print('================ build data only using timestamp ================')
init_train, timestamp = build_init_data_by_ts(train, ratio)
print('timestamp {} for {} training data: '.format(timestamp, ratio))
init_data_stat(init_train, train)

print('\nbuild data using timestamp and make sure each user has at least 5 behaviors')
init_train, timestamp = build_init_data_by_ts_and_k(train, ratio, 5)
print('timestamp {} for {} and init 5 training data: '.format(timestamp, ratio))
init_data_stat(init_train, train)

# full train : 1772571
================ build data only using timestamp ================
timestamp 1418640605 for 0.5 training data: 
# init train : 886285
# data ratio : 0.49999971792385184
# empty user in train number: 4687
max user behavior seq len:  1224
min user behavior seq len:  0
avg user behavior seq len:  46.28120104438642
median user behavior seq len:  30.0
====================== test item coverage =====================
# items in init training data:  46303
test coverage in train:  11670.0 0.6093994778067885
# test items not found in training data:  7480

build data using timestamp and make sure each user has at least 5 behaviors
timestamp 1412507017 for 0.5 and init 5 training data: 
# init train : 886285
# data ratio : 0.49999971792385184
# empty user in train number: 0
max user behavior seq len:  1208
min user behavior seq len:  5
avg user behavior seq len:  46.28120104438642
median user behavior seq len:  27.0
====================== test item coverage ===================

In [7]:
print('# full train :', sum([len(seq) for u, seq in train.items()]))

# 70% init data
ratio = 0.7
print('================ build data only using timestamp ================')
init_train, timestamp = build_init_data_by_ts(train, ratio)
print('timestamp {} for {} training data: '.format(timestamp, ratio))
init_data_stat(init_train, train)

print('\nbuild data using timestamp and make sure each user has at least 5 behaviors')
init_train, timestamp = build_init_data_by_ts_and_k(train, ratio, 5)
print('timestamp {} for {} and init 5 training data: '.format(timestamp, ratio))
init_data_stat(init_train, train)

# full train : 1772571
================ build data only using timestamp ================
timestamp 1486569349 for 0.7 training data: 
# init train : 1240799
# data ratio : 0.6999996050933925
# empty user in train number: 1703
max user behavior seq len:  1294
min user behavior seq len:  0
avg user behavior seq len:  64.793681462141
median user behavior seq len:  47.0
====================== test item coverage =====================
# items in init training data:  52651
test coverage in train:  14363.0 0.7500261096605744
# test items not found in training data:  4787

build data using timestamp and make sure each user has at least 5 behaviors
timestamp 1484502437 for 0.7 and init 5 training data: 
# init train : 1240799
# data ratio : 0.6999996050933925
# empty user in train number: 0
max user behavior seq len:  1291
min user behavior seq len:  5
avg user behavior seq len:  64.793681462141
median user behavior seq len:  47.0
====================== test item coverage =====================
#

In [8]:
print('# full train :', sum([len(seq) for u, seq in train.items()]))

# 10% init data
ratio = 0.1
print('================ build data only using timestamp ================')
init_train, timestamp = build_init_data_by_ts(train, ratio)
print('timestamp {} for {} training data: '.format(timestamp, ratio))
init_data_stat(init_train, train)

print('\nbuild data using timestamp and make sure each user has at least 5 behaviors')
init_train, timestamp = build_init_data_by_ts_and_k(train, ratio, 5)
print('timestamp {} for {} and init 5 training data: '.format(timestamp, ratio))
init_data_stat(init_train, train)

# full train : 1772571
================ build data only using timestamp ================
timestamp 1248352790 for 0.1 training data: 
# init train : 177256
# data ratio : 0.09999937943247407
# empty user in train number: 15228
max user behavior seq len:  517
min user behavior seq len:  0
avg user behavior seq len:  9.256187989556135
median user behavior seq len:  0.0
====================== test item coverage =====================
# items in init training data:  23017
test coverage in train:  5863.0 0.3061618798955614
# test items not found in training data:  13287

build data using timestamp and make sure each user has at least 5 behaviors
timestamp 1215507989 for 0.1 and init 5 training data: 
# init train : 177256
# data ratio : 0.09999937943247407
# empty user in train number: 0
max user behavior seq len:  425
min user behavior seq len:  5
avg user behavior seq len:  9.256187989556135
median user behavior seq len:  5.0
====================== test item coverage =====================
